In [1]:
# feature generation code from: https://github.com/t-davidson/hate-speech-and-offensive-language

In [2]:
import pandas as pd
import numpy as np
import pickle
import sys
#reload(sys)
#sys.setdefaultencoding("utf-8")

#Loading raw data
df = pd.read_csv("./data/hate.csv", delimiter=",", dtype='unicode')

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re

# nltk.download()
stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()


def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = parsed_text.code("utf-8", errors='ignore')
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    #tokens = re.split("[^a-zA-Z]*", tweet.lower())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()

vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords, #We do better when we keep stopwords
    use_idf=True,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=10000,
    min_df=5,
    max_df=0.501
    )

In [4]:
tweets = df.iloc[:,6].values
#Construct tfidf matrix and get relevant scores
tfidf = vectorizer.fit_transform(tweets).toarray()
vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
idf_vals = vectorizer.idf_
idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys are indices; values are IDF scores

/Users/jainulvaghasia/anaconda/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [5]:
#Get POS tags for tweets and save as a string
tweet_tags = []
for t in tweets:
    tokens = basic_tokenize(preprocess(t))
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    #for i in range(0, len(tokens)):
    tag_str = " ".join(tag_list)
    tweet_tags.append(tag_str)
        #print(tokens[i],tag_list[i])

In [6]:
#We can use the TFIDF vectorizer to get a token matrix for the POS tags
pos_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=5000,
    min_df=5,
    max_df=0.501,
    )

In [7]:
#Construct POS TF matrix and get vocab dict
pos = pos_vectorizer.fit_transform(pd.Series(tweet_tags)).toarray()
pos_vocab = {v:i for i, v in enumerate(pos_vectorizer.get_feature_names())}

In [8]:
#Now get other features
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textstat.textstat import *

sentiment_analyzer = SentimentIntensityAnalyzer()

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    ##SENTIMENT
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words) #count syllables in words
    num_chars = sum(len(w) for w in words) #num chars in words
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet) #Count #, @, and http://
    retweet = 0
    if "rt" in words:
        retweet = 1
    features = [FKRA, FRE,syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],
                twitter_objs[2], twitter_objs[1],
                twitter_objs[0], retweet]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)

In [9]:
other_features_names = ["FKRA", "FRE","num_syllables", "avg_syl_per_word", "num_chars", "num_chars_total", \
                        "num_terms", "num_words", "num_unique_words", "vader neg","vader pos","vader neu", "vader compound", \
                        "num_hashtags", "num_mentions", "num_urls", "is_retweet"]

In [10]:
feats = get_feature_array(tweets)

In [11]:
#Now join them all up
M = np.concatenate([tfidf,pos,feats],axis=1)

In [12]:
M.shape

(24783, 11166)

In [13]:
#Finally get a list of variable names
variables = ['']*len(vocab)
for k,v in vocab.iteritems():
    variables[v] = k

pos_variables = ['']*len(pos_vocab)
for k,v in pos_vocab.iteritems():
    pos_variables[v] = k

feature_names = variables+pos_variables+other_features_names

In [14]:
X = pd.DataFrame(M)
y = df['class'].astype(int)
y = map(lambda x : 1 if x == 0 else 0, y)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [16]:
select = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l1",C=0.01))
X_ = select.fit_transform(X,y)

/Users/jainulvaghasia/anaconda/lib/python2.7/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [22]:
model = LinearSVC(class_weight='balanced',C=0.01, penalty='l2', loss='squared_hinge',multi_class='ovr').fit(X_[:17000], y[:17000])

In [23]:
y_preds = model.predict(X_[17000:])

In [25]:
report = classification_report( y[17000:], y_preds )

In [26]:
print(report)

             precision    recall  f1-score   support

          0       0.98      0.93      0.96      7442
          1       0.30      0.62      0.41       341

avg / total       0.95      0.92      0.93      7783



In [31]:
from sklearn import metrics
y_preds = model.decision_function(X_[17000:])
fpr, tpr, thresholds = metrics.roc_curve(y[17000:], y_preds, pos_label=0)
print metrics.roc_auc_score(y[17000:], y_preds)
a = [1 for i in range(len(y_preds))]
metrics.roc_auc_score(y[17000:], a)

0.8831124922272809


0.5

In [28]:
def classify(X, w, b):
    return np.sign(b + np.dot(X, w))

def err(y, yhat):
    return np.count_nonzero(np.subtract(y, yhat)) / np.float(len(yhat))

def log_loss(X, y, w, b, lam):
    sig = np.exp(-y * (b + np.dot(X, w)))
    obj = np.mean(np.log(1. + sig)) + lam * np.linalg.norm(w, 2)**2
    return obj

def d_log_loss(X, y, w, b, lam):
    grad = np.empty(np.shape(w)[0]+1)
    sig = np.exp(-y * (b + np.dot(X,w)))
    mu = sig / (1. + sig)
    c = y * mu
    grad[0] = -np.mean(c)
    grad[1:] = -np.mean(c * X.T, axis=1) + 2 * lam * w
    return grad


def d2_log_loss(X, y, w, b, lam):
    b_grad = 0.
    w_grad = np.matrix((np.shape(w)[0], np.shape(w)[0]))
    mu = 1. / (1. + np.exp(-y * (b + np.dot(X,w))))
    sig = mu * (1. - mu)
    c = y * y * sig
    n = len(y)
    b_grad = (1. / n) * np.sum(c)

    k = np.dot(X.T, (c * X.T).T)
    w_grad = (1. / n) * k + 2 * lam * np.eye(np.shape(w)[0])
    return b_grad, w_grad


def gradient_descent(X, y, test_in, test_out,
                      lamb= 1.e-2,
                      max_iter=15000,
                      batch_size=None):
    w = np.zeros(np.shape(X)[1])
    b = 0
    eta = 1.e+0

    train_err = []
    test_err = []
    train_obj = []
    test_obj = []

    for _ in range(max_iter):
      if batch_size:
        ind = np.random.choice(range(len(y)), size = batch_size)
        batch_x, batch_y = X[ind], y[ind]
      else:
        batch_x, batch_y = X, y

      print _
      gradient = d_log_loss(batch_x, batch_y, w, b, lamb)
      w -= eta * gradient[1:]
      b -= eta * gradient[0]

      training_log_obj = log_loss(X, y, w, b, lamb)
      train_obj.append(training_log_obj)
      print training_log_obj

#       test_log_obj = log_loss(test_in, test_out, w, b, lamb)
#       test_obj.append(test_log_obj)
#       print test_log_obj

#       yhat = classify(X, w, b)
#       training_err = err(yhat, y)
#       train_err.append(training_err)
#       print training_err

#       test_outhat = classify(test_in, w, b)
#       testing_err = err(test_outhat, test_out)
#       test_err.append(testing_err)
#       print testing_err

      print '----------------------------------------'

    return b, w
    #return train_err, test_err, train_obj, test_obj

In [29]:
y = np.array(y)
b0, w0 = gradient_descent(X_[:17000], y[:17000], X_[17000:], y[17000:])

0
0.9010926327806714
----------------------------------------
1
0.884336748838803
----------------------------------------
2
0.8684352651247722
----------------------------------------
3
0.8539700992743359
----------------------------------------
4
0.8402035247300387
----------------------------------------
5
0.8273355602914345
----------------------------------------
6
0.815382026317374
----------------------------------------
7
0.80505024161787
----------------------------------------
8
0.7957849362212718
----------------------------------------
9
0.7875080804781138
----------------------------------------
10
0.78044064612362
----------------------------------------
11
0.7740191935324384
----------------------------------------
12
0.7679411111181424
----------------------------------------
13
0.762120700644726
----------------------------------------
14
0.7565337124385878
----------------------------------------
15
0.7511684797776407
----------------------------------------
16
0.7460

0.6495415902201986
----------------------------------------
135
0.6495121904937626
----------------------------------------
136
0.6494840473217172
----------------------------------------
137
0.6494571173305098
----------------------------------------
138
0.6494313595547543
----------------------------------------
139
0.6494067354049694
----------------------------------------
140
0.649383208634955
----------------------------------------
141
0.6493607453056757
----------------------------------------
142
0.6493393137414923
----------------------------------------
143
0.6493188844733835
----------------------------------------
144
0.6492994301624461
----------------------------------------
145
0.6492809254956009
----------------------------------------
146
0.6492633470442398
----------------------------------------
147
0.649246673075802
----------------------------------------
148
0.6492308833084299
----------------------------------------
149
0.6492159586004216
-----------------------

0.6490256423879924
----------------------------------------
266
0.649025616953468
----------------------------------------
267
0.649025592224918
----------------------------------------
268
0.6490255681716518
----------------------------------------
269
0.6490255447643148
----------------------------------------
270
0.6490255219748279
----------------------------------------
271
0.6490254997763324
----------------------------------------
272
0.6490254781431386
----------------------------------------
273
0.6490254570506724
----------------------------------------
274
0.6490254364754292
----------------------------------------
275
0.6490254163949265
----------------------------------------
276
0.6490253967876591
----------------------------------------
277
0.6490253776330573
----------------------------------------
278
0.649025358911446
----------------------------------------
279
0.6490253406040057
----------------------------------------
280
0.6490253226927353
------------------------

0.6490240669285093
----------------------------------------
397
0.6490240577201197
----------------------------------------
398
0.6490240485142503
----------------------------------------
399
0.6490240393108109
----------------------------------------
400
0.6490240301097147
----------------------------------------
401
0.6490240209108789
----------------------------------------
402
0.6490240117142241
----------------------------------------
403
0.649024002519675
----------------------------------------
404
0.6490239933271583
----------------------------------------
405
0.6490239841366054
----------------------------------------
406
0.6490239749479498
----------------------------------------
407
0.649023965761128
----------------------------------------
408
0.649023956576079
----------------------------------------
409
0.649023947392745
----------------------------------------
410
0.6490239382110706
----------------------------------------
411
0.6490239290310023
-------------------------

0.6490228422136631
----------------------------------------
531
0.6490228331114208
----------------------------------------
532
0.6490228240096171
----------------------------------------
533
0.6490228149082515
----------------------------------------
534
0.6490228058073241
----------------------------------------
535
0.6490227967068342
----------------------------------------
536
0.6490227876067819
----------------------------------------
537
0.6490227785071668
----------------------------------------
538
0.649022769407989
----------------------------------------
539
0.649022760309248
----------------------------------------
540
0.6490227512109434
----------------------------------------
541
0.6490227421130754
----------------------------------------
542
0.6490227330156438
----------------------------------------
543
0.6490227239186481
----------------------------------------
544
0.6490227148220886
----------------------------------------
545
0.6490227057259648
-----------------------

...

0.648929013628323
----------------------------------------
14411
0.6489290086807823
----------------------------------------
14412
0.648929003733442
----------------------------------------
14413
0.6489289987863022
----------------------------------------
14414
0.6489289938393626
----------------------------------------
14415
0.6489289888926236
----------------------------------------
14416
0.6489289839460847
----------------------------------------
14417
0.6489289789997463
----------------------------------------
14418
0.6489289740536082
----------------------------------------
14419
0.6489289691076703
----------------------------------------
14420
0.6489289641619327
----------------------------------------
14421
0.6489289592163956
----------------------------------------
14422
0.6489289542710587
----------------------------------------
14423
0.648928949325922
----------------------------------------
14424
0.6489289443809857
----------------------------------------
14425
0.64892893943

0.6489283918129003
----------------------------------------
14537
0.6489283868905271
----------------------------------------
14538
0.6489283819683533
----------------------------------------
14539
0.6489283770463784
----------------------------------------
14540
0.6489283721246026
----------------------------------------
14541
0.6489283672030258
----------------------------------------
14542
0.6489283622816482
----------------------------------------
14543
0.6489283573604694
----------------------------------------
14544
0.6489283524394898
----------------------------------------
14545
0.6489283475187091
----------------------------------------
14546
0.6489283425981274
----------------------------------------
14547
0.6489283376777448
----------------------------------------
14548
0.6489283327575612
----------------------------------------
14549
0.6489283278375765
----------------------------------------
14550
0.6489283229177908
----------------------------------------
14551
0.64892831

0.648927773158359
----------------------------------------
14663
0.6489277682609901
----------------------------------------
14664
0.6489277633638189
----------------------------------------
14665
0.6489277584668455
----------------------------------------
14666
0.6489277535700698
----------------------------------------
14667
0.6489277486734918
----------------------------------------
14668
0.6489277437771116
----------------------------------------
14669
0.6489277388809294
----------------------------------------
14670
0.6489277339849446
----------------------------------------
14671
0.6489277290891577
----------------------------------------
14672
0.6489277241935685
----------------------------------------
14673
0.6489277192981769
----------------------------------------
14674
0.648927714402983
----------------------------------------
14675
0.648927709507987
----------------------------------------
14676
0.6489277046131884
----------------------------------------
14677
0.64892769971

0.6489271478993109
----------------------------------------
14791
0.6489271430271766
----------------------------------------
14792
0.6489271381552387
----------------------------------------
14793
0.6489271332834976
----------------------------------------
14794
0.6489271284119527
----------------------------------------
14795
0.6489271235406044
----------------------------------------
14796
0.6489271186694525
----------------------------------------
14797
0.6489271137984969
----------------------------------------
14798
0.648927108927738
----------------------------------------
14799
0.6489271040571752
----------------------------------------
14800
0.6489270991868089
----------------------------------------
14801
0.6489270943166392
----------------------------------------
14802
0.6489270894466659
----------------------------------------
14803
0.6489270845768889
----------------------------------------
14804
0.648927079707308
----------------------------------------
14805
0.6489270748

0.6489265355544603
----------------------------------------
14917
0.6489265307070041
----------------------------------------
14918
0.6489265258597433
----------------------------------------
14919
0.6489265210126777
----------------------------------------
14920
0.648926516165807
----------------------------------------
14921
0.6489265113191318
----------------------------------------
14922
0.6489265064726516
----------------------------------------
14923
0.6489265016263666
----------------------------------------
14924
0.6489264967802767
----------------------------------------
14925
0.6489264919343821
----------------------------------------
14926
0.6489264870886825
----------------------------------------
14927
0.6489264822431782
----------------------------------------
14928
0.6489264773978689
----------------------------------------
14929
0.6489264725527546
----------------------------------------
14930
0.6489264677078356
----------------------------------------
14931
0.648926462

In [30]:
y_preds = classify(X_[17000:], w0, b0)
from sklearn.metrics import f1_score
print f1_score(y[17000:], y_preds, average='weighted')

0.003678085397778604


/Users/jainulvaghasia/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [33]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_[:17000], y[:17000])

GaussianNB(priors=None)

In [35]:
y_preds = clf.predict(X_[17000:])

In [37]:
report = classification_report( y[17000:], y_preds )
print report

             precision    recall  f1-score   support

          0       0.98      0.89      0.93      7442
          1       0.20      0.60      0.30       341

avg / total       0.95      0.87      0.90      7783

